# Extração da Drogaria Minas Mais

In [7]:
import requests
from bs4 import BeautifulSoup
import os
import time
import json
import pandas as pd

/tmp/ipykernel_278908/62150564.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [15]:
# Capturando todas as URLs

# Função de acessar site
all_workers = os.cpu_count()

def access_site(url, max_attempts=all_workers):
    headers = {'Accept': '*/*', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    for retry in range(max_attempts):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
        except Exception as e:
                print(f"An error occurred while accessing URL: {url}: {e}. Retrying... (Attempt {retry + 1}/{max_attempts})")
                time.sleep(1)

# Função captura de todos os nomes de marcas e criação de URLs
def getNomeMarcas():
    response = access_site("https://www.drogariasminasmais.com.br/medicamentos")
    soup = BeautifulSoup(response.text, 'html.parser')
    sessaoMarcasGrandes = soup.find('div', class_='vtex-search-result-3-x-filter__container vtex-search-result-3-x-filter__container--responsive-result-content-mz0001 bb b--muted-4 vtex-search-result-3-x-filter__container--brand')
    sessaoMarcas = sessaoMarcasGrandes.find('div', class_='vtex-search-result-3-x-filterTemplateOverflow vtex-search-result-3-x-filterTemplateOverflow--responsive-result-content-mz0001 pb5 overflow-y-auto')
    marcas = sessaoMarcas.find_all('label', class_='vtex-checkbox__label w-100 c-on-base pointer')
    return marcas

# Criando uma lista de URLs ainda não bem estruturadas
urlSuja = getNomeMarcas()
urlLimpa = []

# Limpando lista das URLs
for i in urlSuja:
    # Criando texto separado e minusculo com "-"
    texto = i.text.lower()
    textoNovo = texto.replace(" ", "-")
    textoNovissimo = textoNovo.replace("/", "-")

    # Salvando tudo em uma lista
    urlLimpa.append(textoNovissimo)

listaUrls = []

# Criando todas as URLs
for j in urlLimpa:
    URL = "https://www.drogariasminasmais.com.br/medicamentos/" + j + "?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page="
    listaUrls.append(URL)

In [84]:
# Achando número de produtos
url = "https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1"
requisicao = requests.get(url)
soup = BeautifulSoup(requisicao.text, 'html.parser')
numeroProdutos = soup.find_all('script')[-34].text
numeroProdutos = numeroProdutos[:-8].split("quantity")[66:]

listaNumeroProdutos = []
listaNumeroProdutosCorreta = []

for i in numeroProdutos:
    listaNumeroProdutos.append(i.split(",")[0])

for i in listaNumeroProdutos:
    aux = i.replace('"', "")
    aux = aux.replace(":", "")
    listaNumeroProdutosCorreta.append(int(aux))

In [85]:
def getJSON(soup):
    jsonSujo = soup.find_all('script')[-34].text
    jsonSujo = jsonSujo[:-8]
    jsonLimpo = jsonSujo.split("__STATE__ = ")[1]
    return jsonLimpo

total_pages = []

for i in listaNumeroProdutosCorreta:
    total_pages.append((i//15) + 1)

listaJSON = []

for i in listaUrls: 
    for page_number in total_pages:
        for paginaAtual in range(1, page_number):
            url = str(i) + str(paginaAtual)
            requisicao = requests.get(url)
            soup = BeautifulSoup(requisicao.text, 'html.parser')
            listaJSON.append(getJSON(soup))

In [31]:
# Limpando JSON
url = "https://www.drogariasminasmais.com.br/medicamentos/cimed?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=12"
requisicao = requests.get(url)
soup = BeautifulSoup(requisicao.text, 'html.parser')
jsonSujo = soup.find_all('script')[-34].text
jsonSujo = jsonSujo[:-8]
jsonLimpo = jsonSujo.split("__STATE__ = ")[1]
print(jsonLimpo)

{"Product:sp-15742":{"cacheId":"sp-15742","productId":"15742","description":"Nevralgex é indicado no alívio da dor associada a contraturas musculares, incluindo dor de cabeça tensional.","productName":"Nevralgex com 10 Comprimidos - Cimed","productReference":"8483","linkText":"nevralgex-com-10-comprimidos---cimed","brand":"CIMED","brandId":137,"link":"\u002Fnevralgex-com-10-comprimidos---cimed\u002Fp","categories":{"type":"json","json":["\u002FMedicamentos\u002FRelaxante Muscular\u002F","\u002FMedicamentos\u002F"]},"categoryId":"16","releaseDate":"1715018520000","priceRange":{"type":"id","generated":true,"id":"$Product:sp-15742.priceRange","typename":"ProductPriceRange"},"specificationGroups":[{"type":"id","generated":true,"id":"Product:sp-15742.specificationGroups.0","typename":"SpecificationGroup"},{"type":"id","generated":true,"id":"Product:sp-15742.specificationGroups.1","typename":"SpecificationGroup"}],"skuSpecifications":[],"productClusters":[],"clusterHighlights":[],"properties

In [66]:
# Criando o JSON e capturando todos os códigos de produtos
ean = []
precoComDesconto = []
precoSemDesconto = []
desconto = []
marcas = []

for k in listaJSON:
    dados = json.loads(k)
    listaProdutos = []
    listaProdutosCerta = []
    listaProdutosSuperCerta = []
    j = 0 

    for i in dados.keys():
        if j % 18 == 0:
            listaProdutos.append(i)
        j = j + 1
        if j == 270:
            break

    for i in listaProdutos:
        listaProdutosCerta.append(i.split(".")[0])

    for i in listaProdutosCerta:
        listaProdutosSuperCerta.append(i.replace("$", ""))
    
    # Pegando todos os dados dos produtos
    for i in listaProdutosSuperCerta:
        marcas.append(dados[i]["brand"])

    for i in listaProdutosSuperCerta:
        # Parte do JSON que tem o EAN e precos
        localJson1 = str(i) + '.items({"filter":"ALL"}).0'
        dadosDesejados1 = dados.get(localJson1)

        ean.append(dadosDesejados1["ean"])

    for i in listaProdutosSuperCerta:
        # Parte do JSON que tem o EAN e precos
        localJson2 = "$" + str(i) + '.items({"filter":"ALL"}).0.sellers.0.commertialOffer'
        dadosDesejados2 = dados.get(localJson2)

        precoComDesconto.append(dadosDesejados2["Price"])
        precoSemDesconto.append(dadosDesejados2["ListPrice"])
        desconto.append((1 - (dadosDesejados2["Price"]/dadosDesejados2["ListPrice"])))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
data = {"marcas": marcas, "EAN": ean, "Preço com desconto": precoComDesconto, "Preço sem desconto": precoSemDesconto, "Desconto": desconto}
df = pd.DataFrame(data)

In [21]:
# Exportando tudo para um csv
df.to_excel('DadosMinasMais.xlsx',sheet_name='Sheet1')